table of contents
1. 関数

Juliaの特徴
* コンパイル言語に比肩する実行速度
* シンプルな言語機能と豊富な標準ライブラリ
* 引数の型により実行される関数が決まる動的ディスパッチ
* Lispのような強力なマクロ機能

# 関数

まずはJuliaのコードをざっと見てみましょう。配列をソートするクイックソートのコードです。

In [1]:
quicksort(xs) = quicksort!(copy(xs))
quicksort!(xs) = quicksort!(xs, 1, length(xs))

function quicksort!(xs, lo, hi)
    if lo < hi
        p = partition(xs, lo, hi)
        quicksort!(xs, lo, p - 1)
        quicksort!(xs, p + 1, hi)
    end
    return xs
end

function partition(xs, lo, hi)
    pivot = div(lo + hi, 2)
    pvalue = xs[pivot]
    xs[pivot], xs[hi] = xs[hi], xs[pivot]
    j = lo
    @inbounds for i in lo:hi-1
        if xs[i] <= pvalue
            xs[i], xs[j] = xs[j], xs[i]
            j += 1
        end
    end
    xs[j], xs[hi] = xs[hi], xs[j]
    return j
end

partition (generic function with 1 method)

どうでしょう。PythonやRubyをやったことがる人なら初見でも大体の意味が分かるのではないでしょうか。
まるで擬似コードのような、スッキリした文法です。

関数定義・分岐・反復などの構文はそれぞれ`function ... end`, `if ... end`, `for ... end`, `while ... end`のようにキーワードで始まり`end`で終わります。 ちょうどRubyと同じような感じです。 インデントはPythonのように必要ではありませんが、4スペースでひとつのインデントを表すのが慣習です。

また、6行目の`p = partition(xs, lo, hi)`で分かるように変数の宣言や型の指定は通常必要ありません。

18行目の`@inbounds`というのが気になるかもしれません。
`@`マークで始まるこの表記は**マクロ呼出し**と言われ、コードを書き換えたりJuliaのコンパイラに最適化のヒントを与えることができます。
ここで使われている`@inbounds`は添字アクセス(`xs[i]`など)のチェックを省き、少し計算を高速化できますが、配列の範囲外にアクセスした時はセグメンテーション違反などを起こし停止する可能性があります。

こうしたJuliaで書かれたコードはJuliaのLLVMベースのJITコンパイラによりコンパイルされ、C言語などで書いたコードとそれほど変わらない速度で実行できます。

試しに整数の小さい配列をソートしてみると、うまく行っています。

In [2]:
quicksort([3, 6, 2, 4, 5, 1])

6-element Vector{Int64}:
 1
 2
 3
 4
 5
 6

浮動小数点でも動きます。

In [3]:
quicksort([-2.1, 3.4, 5.0, -1.2, 3.1, 0.1])

6-element Vector{Float64}:
 -2.1
 -1.2
  0.1
  3.1
  3.4
  5.0

1千万要素もあるような浮動小数点数の配列のソートも一瞬です。

In [4]:
using Random: seed!
seed!(0xdeadbeef)
xs = randn(10_000_000)
@time quicksort(xs)

  1.177913 seconds (2 allocations: 76.294 MiB, 0.70% gc time)


10000000-element Vector{Float64}:
 -5.195847643339796
 -4.965462660891118
 -4.9061968556410385
 -4.853960867689443
 -4.780966591018771
 -4.75993935971095
 -4.75766633895526
 -4.715715468600939
 -4.708323665744881
 -4.704676339449706
 -4.677681840439496
 -4.65721080889134
 -4.653060310117849
  ⋮
  4.633787920161552
  4.651267150552577
  4.6583917283576
  4.733092376976452
  4.747430603222531
  4.78254242404567
  4.833569207548429
  4.844964859905789
  4.893076903998649
  4.960872359435442
  5.063606724060488
  5.110565738696973

ここでひとつ注目すべきことは、`quicksort`関数の定義時に引数の型を指定していなかったにも関わらず、整数にも浮動小数点数にも適用できるということです。
実は、関数の最初の実行時にそれぞれの型に応じて高速なコードを生成しています。
この機能のおかげで、**Juliaでは関数の型を一切指定しなくても十分なパフォーマンスが得られます**。

`quicksort`は、数値にかぎらず以下の様な文字や文字列でも適用できます。

In [5]:
quicksort(['B', 'A', 'D', 'E', 'C'])

5-element Vector{Char}:
 'A': ASCII/Unicode U+0041 (category Lu: Letter, uppercase)
 'B': ASCII/Unicode U+0042 (category Lu: Letter, uppercase)
 'C': ASCII/Unicode U+0043 (category Lu: Letter, uppercase)
 'D': ASCII/Unicode U+0044 (category Lu: Letter, uppercase)
 'E': ASCII/Unicode U+0045 (category Lu: Letter, uppercase)

In [6]:
quicksort(["Bob", "Alice", "Dave", "Eve", "Charlie"])

5-element Vector{String}:
 "Alice"
 "Bob"
 "Charlie"
 "Dave"
 "Eve"

## 関数とメソッド

### 関数の定義

既に何度か出てきますが、関数の定義は

```julia
function <関数名>(<引数>, ...)
    <関数本体>
end
```

を使います。
返り値は`return`を使いますが、最後に評価された式が自動的に返り値になるので省略可能です。

In [7]:
# 2次元空間でのpとq間のユークリッド距離
function dist(p, q)
    dx = p[1] - q[1]
    dy = p[2] - q[2]
    sqrt((dx)^2 + (dy)^2)
end

dist (generic function with 1 method)

In [8]:
dist((0, 0), (3, 4))

5.0

もう一つ、別の方法として以下の「代入」のような形も使えます。一行で済むような
簡単な関数を定義するときに便利です。

In [9]:
dist(p, q) = sqrt((p[1] - q[1])^2 + (p[2] - q[2])^2)

dist (generic function with 1 method)

In [10]:
dist((0, 0), (3, 4))

5.0

オプション引数やキーワード引数、可変長引数もJuliaでは使えます。

In [11]:
# オプション引数: '='
ktop(xs, k=3) = sort(xs)[1:k]

ktop (generic function with 2 methods)

In [12]:
ktop([1,5,3,2,6])

3-element Vector{Int64}:
 1
 2
 3

In [13]:
ktop([1,5,3,2,6], 2)

2-element Vector{Int64}:
 1
 2

In [14]:
# キーワード引数: ';' '='
function ktop(xs; k=3, rev=false)
    sort(xs, rev=rev)[1:k]
end

ktop (generic function with 2 methods)

In [15]:
ktop([1,5,3,2,6], k=2)

2-element Vector{Int64}:
 1
 2

In [16]:
ktop([1,5,3,2,6], k=4, rev=true)

4-element Vector{Int64}:
 6
 5
 3
 2

In [17]:
function pathlength(p, q, rs...)
    len = dist(p, q)
    for r in rs
        len += dist(q, r)
        q = r
    end
    return len
end

pathlength (generic function with 1 method)

In [18]:
pathlength((0, 0), (1, 1))

1.4142135623730951

In [19]:
pathlength((0, 0), (1, 1), (1, 2))

2.414213562373095

In [20]:
pathlength((0, 0), (1, 1), (1, 2), (0, 0))

4.650281539872885

それぞれに特徴的な記号をまとめておきます。

<table>
    <tr><th>オプション引数</th><td>`=`</td><td>`xs, k=3`</td></tr>
    <tr><th>キーワード引数</th><td>`;` `=`</td><td>`xs; k=3`</td></tr>
    <tr><th>可変長引数</th><td>`...`</td><td>`x, xs...`</td></tr>
</table>

ここで、Juliaでの関数名の慣習を確認しておきます。
ドキュメントや標準ライブラリの関数から推奨される関数の命名法は以下の様なものになります
(括弧内は標準ライブラリにある関数の具体例です)。

* すべて小文字 (`get`, `readline`)
* アンダースコア(`_`)は、なるべく使わない (`readdlm`, `findmin`)
* 行う操作を表す英語の動詞を使う (`open`, `serialize`)
* `Bool`を返す関数は`is...`や`has...`という名前を使う (`isempty`, `haskey`)
* 引数のデータを変えてしまう関数は最後に`!`をつける (`push!`, `sort!`)

一番最初の例を振り返ってみると、この形の関数定義を使っていました。

```julia
quicksort(xs) = quicksort!(copy(xs))
quicksort!(xs) = quicksort!(xs, 1, length(xs))
```

`quicksort!`のように、関数名の最後に`!`をつけるのは引数を変更するときの慣習的な方法です。
関数の引数は基本的に参照渡しのようになるため、`Array`などmutableな型の値は関数内で変更できます。

### 多重ディスパッチ

実はJuliaでは、関数がとる引数の型を`<引数>::<型>`のように指定することもできます。これは他のスクリプト言語にはあまりみられない特徴です。

In [21]:
function add(x::Int, y::Int)
    return x + y
end

add (generic function with 1 method)

型が合えば関数を呼び出すことができます。

In [22]:
add(1, 2)

3

しかし型が合わないと、呼び出すこともできません。

In [23]:
add(1.0, 2.0)

LoadError: MethodError: no method matching add(::Float64, ::Float64)

ここで、次の関数を定義してみましょう

In [24]:
function add(x::Float64, y::Float64)
    return x + y
end

add (generic function with 2 methods)

すると、先ほど呼び出せなかった方の組み合わせで呼び出せるようになります

In [25]:
add(1.0, 2.0)

3.0

しかも、これは元の関数の上書きではありません。元の`Int`, `Int`の組み合わせでも呼び出せます。

In [26]:
add(1, 2)

3

このように、引数の型に合わせて呼び出すメソッドを変える仕組みを**多重ディスパッチ**と呼びます。
以下の例で、これがどのように動くのかを確認して下さい。

In [27]:
function foo(x::Int)
    println("foo Int: $x")
end

function foo(x::Int, y::Int)
    println("foo Int Int: $x $y")
end

function foo(x::Float64, y::Float64)
    println("foo Float64 Float64: $x $y")
end

function foo(x::Int, y::Float64)
    println("foo Int Float64: $x $y")
end 

foo (generic function with 4 methods)

In [28]:
foo(1)

foo Int: 1


In [29]:
foo(1, 2)

foo Int Int: 1 2


In [30]:
foo(1.0, 3.14)

foo Float64 Float64: 1.0 3.14


In [31]:
foo(1, 3.14)

foo Int Float64: 1 3.14


型の指定は`Int64`や`Float64`のような具体型に限りません。
より高位の抽象型を使うこともできます。

In [32]:
add(BigInt(1), BigFloat(1))  # これにマッチするメソッドはまだない

LoadError: MethodError: no method matching add(::BigInt, ::BigFloat)

`x isa Number`かつ`y isa Number`になれば、以下のメソッドを呼び出せますので、具体型を使っていたときに比べ適用範囲がぐっと広がります。

In [33]:
function add(x::Number, y::Number)
    return x + y
end

add (generic function with 3 methods)

In [34]:
add(BigInt(1), BigFloat(1))  # 今度は呼び出せる

2.0

In [35]:
add(1, π), add(π, 1.0), add(true, false)  # 様々な型の組み合わせが使える  

(4.141592653589793, 4.141592653589793, 1)

### コンストラクタ

Juliaのコンストラクタは通常の関数と同様に定義でき、多重ディスパッチも利用できます。
コンストラクタは以下の2種類に分けられます。

* 内部コンストラクタ (inner constructor)
* 外部コンストラクタ (outer constructor)

外部コンストラクタは、他の外部コンストラクタや内部コンストラクタを呼び出すことでオブジェクトを作ることができます。
最終的に、全てのオブジェクトは内部コンストラクタを経由して作られるため、**内部コンストラクタで最終的な値のチェック**などを実現できます。

それでは、具体例を見てみましょう。

以下の`RGB`型はRed-Green-Blueの3原色を指定して色を表現する型です。
それぞれの色は8bitの符号なし整数でエンコーディングしています。
しかし、色を作る際は8bitで表現できない値が与えられる可能性があるため、不正な値が与えられれば例外を投げるようにしたいです。
これを実現するため、内部コンストラクタで与えられた値のチェックをしています。
内部コンストラクタは、構文的には`(mutable) struct ... end`の内部で定義された関数です。
内部コンストラクタの特別な点は、`new`関数を呼び出すことで、その型のオブジェクトを作ることができるところにあります。

In [36]:
struct RGB
    r::UInt8
    g::UInt8
    b::UInt8

    function RGB(r, g, b)
        @assert 0 <= r <= 255
        @assert 0 <= g <= 255
        @assert 0 <= b <= 255
        return new(r, g, b)
    end
end

In [37]:
RGB(1, 2, 3)

RGB(0x01, 0x02, 0x03)

300など8bitで表現できない色が与えられたら、`@assert`マクロが例外を投げます。

In [38]:
RGB(2, 300, 2)

LoadError: AssertionError: 0 <= g <= 255

デフォルトコンストラクタは`new`関数を呼び出すだけの内部コンストラクタで、プログラマが明示的に内部コンストラクタを定義するとデフォルトコンストラクタは作られません。

ある1つの値を与えたら、R,G,Bすべてに同じ値を設定する簡便なコンストラクタが欲しいかもしれません。
その場合は以下のように外部コンストラクタを使うと便利です。

In [39]:
RGB(x) = RGB(x, x, x)

RGB

外部コンストラクタは構文的には`(mutable) struct ... end`の外部で定義された関数です。ここでは`new`関数は使えず、内部コンストラクタや他の外部コンストラクタを呼び出すことでオブジェクトを作ります。

これは、他の外部コンストラクタを呼び出す外部コンストラクタの例です。

In [40]:
RGB() = RGB(0)

RGB

多重ディスパッチのお陰で、以下の３つのコンストラクタはすべて使うことができます。

In [41]:
RGB(), RGB(10), RGB(10, 20, 30)

(RGB(0x00, 0x00, 0x00), RGB(0x0a, 0x0a, 0x0a), RGB(0x0a, 0x14, 0x1e))